In [3]:
import torch
import numpy
import pandas

arr = ["CCNN", "EEG", "TSC"]
mode = arr[2]

In [4]:
import time
start = time.time()
# 对训练数据细分
def split_easy(x, y, sign, gap = 16):
    x_out, y_out = None, []
    if sign == 1:
        index = list(pandas.read_csv("../Trail_index.csv")['Test_1'])[:15360]
    else:
        index = list(pandas.read_csv("../Trail_index.csv")['Test_2'])[:15360]

    # index = list(pandas.read_csv("../Random_index.csv")['Test'])[:15360]

    for i in range(gap):
        temp = None
        for j in range(960):
            k = index[i * 960 + j]
            if temp == None:
                temp = torch.unsqueeze(x[k], dim = 0).cuda()
            else:
                temp = torch.cat((torch.unsqueeze(x[k], dim = 0).cuda(), temp), 0).cuda()

            y_out.append(int(y[k]) == 1)

        if i == 0:
            x_out = temp
        else:
            x_out = torch.cat((x_out, temp), 0).cuda()

    return x_out, torch.tensor(y_out), index

x = torch.tensor(numpy.load(f"./x_{mode}.npy", allow_pickle=True))
y = torch.tensor(numpy.load(f"./y_{mode}.npy", allow_pickle=True))

if mode == "CCNN":
    x_train, y_train, index = split_easy(x, y, 1)
else:
    x_train, y_train, index = split_easy(x, y, 2)

x_train = x_train.reshape(x_train.shape[0], x_train.shape[2], -1)
y_train = y_train.reshape(y_train.shape[0])

numpy.save("./train_x.npy", x_train.detach().cpu().numpy())
numpy.save("./train_y.npy", y_train.detach().cpu().numpy())


In [5]:
x = torch.tensor(numpy.load(f"./train_x.npy", allow_pickle=True))
y = torch.tensor(numpy.load(f"./train_y.npy", allow_pickle=True))

x, y = x.reshape(x.shape[0], -1), y
print(x.shape)
print(y.shape)

torch.Size([15360, 3584])
torch.Size([15360])


In [6]:
# SVM
import sklearn.svm
from sklearn.preprocessing import StandardScaler
s = StandardScaler()

model = sklearn.svm.SVC(C = 1, kernel = 'poly', probability = True, max_iter = 150)
model.fit(torch.tensor(s.fit_transform(numpy.array(x))), y)

model.score(x, y)

d:\anaconda\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=150).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.5383463541666667

In [7]:
out = model.predict_proba(x)

arr = []
for i in range(len(out)):
    arr.append((index[i], max(out[i][0], out[i][1])))

index, rate = [], []
arr = sorted(arr, key = lambda x : abs(x[1]))

for i in arr:
    index.append(i[0])
    rate.append(i[1])

pandas.DataFrame({'index':index, 'rate':rate}).to_csv(f"../SVM_{mode}_index.csv", index = False)

print(time.time - start)

TypeError: unsupported operand type(s) for -: 'builtin_function_or_method' and 'float'